## Setup

In [1]:
import os
import json
import glob
import torch
import re
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
#import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display, clear_output, HTML
from utils.data_processing import (
    load_edge_scores_into_dictionary,
    compute_ged,
    compute_weighted_ged,
    compute_gtd,
    compute_jaccard_similarity_to_reference,
    compute_jaccard_similarity,
    compute_weighted_jaccard_similarity,
    compute_weighted_jaccard_similarity_to_reference
)

In [2]:
# =============================================================================
#import kaleido
#pio.renderers.default = 'png' # USE IF MAKING GRAPHS FOR NOTEBOOK EXPORT
# =============================================================================

In [3]:
TASK = 'ioi'
PERFORMANCE_METRIC = 'logit_diff'
MODEL_NAME = 'pythia-160m'

### Plotting Code

In [4]:
def plot_graph_metric(df, metric, perf_metric_dict, title, y_range, x_axis_col='checkpoint', log_x=True):
    # Add a new column for the performance metric by mapping the checkpoint_2 values using the perf_metric_dict
    df['perf_metric'] = df[x_axis_col].map(perf_metric_dict)

    # Interpolate missing values
    df['perf_metric'] = df['perf_metric'].interpolate(method='linear')

    # plot weighted additions, deletions, and total weighted GED over time
    fig = px.line(df, width=1200, x=x_axis_col, y=[metric], title=title, log_x=log_x)

    # Specify colors for each line
    colors = {metric: 'lightblue', 'perf_metric': 'black'}

    # Update each trace with the specified color
    for i, trace in enumerate(fig.data):
        fig.data[i].update(line=dict(color=colors[trace.name]))


    # Convert to a go.Figure to add secondary Y-axis features and add the performance metric line
    fig.update_layout(
        yaxis=dict(range=[0, y_range], title=metric),
        yaxis2=dict(range=[0, 5], title="Logit Difference", overlaying="y", side="right", showgrid=False)
    )
    fig.add_trace(
        go.Scatter(x=df[x_axis_col], y=df['perf_metric'], name='perf_metric', mode='lines', yaxis='y2', line=dict(color=colors['perf_metric']))
    )

    # Optional: Update the layout if you need to adjust titles or other aesthetics
    fig.update_layout(
        yaxis_title=title
    )

    fig.show()

def load_graphs_for_models(target_directory, TASK):
    df_list = []
    for root, dirs, files in os.walk(target_directory):
        for dir in dirs:
            folder_path = f'results/graphs/{dir}/{TASK}'
            if os.path.isdir(folder_path):
                df = load_edge_scores_into_dictionary(folder_path)
                df['subfolder'] = dir  # Add the subfolder name as a new column
                df_list.append(df)
                clear_output()

    combined_df = pd.concat(df_list, ignore_index=True)
    return combined_df

## Retrieve & Process Data

### Circuit Data

In [12]:
# only needs to be run if another model is added or graph files change in some way
all_graphs = load_graphs_for_models('results/graphs', TASK)
all_graphs = all_graphs[all_graphs['in_circuit'] == True]
all_graphs.to_pickle(f'results/all_minimal_graphs_{TASK}.pkl')

In [13]:
# load dataframe from pickle file
all_graphs = pd.read_pickle(f'results/all_minimal_graphs_{TASK}.pkl')
all_graphs = all_graphs[all_graphs['checkpoint'] >= 4000]
all_graphs.rename(columns={'subfolder': 'model'}, inplace=True)
all_graphs.sort_values(by=['model', 'checkpoint'], inplace=True)

all_graphs.head()

,edge,score,in_circuit,checkpoint,model
48588208,m1->m6,0.000349,True,4000,pythia-160m
48588319,a4.h11->a6.h6<v>,0.000355,True,4000,pythia-160m
48588428,a4.h8->a6.h6<v>,0.000450,True,4000,pythia-160m
48588655,m4->m6,0.000431,True,4000,pythia-160m
48589427,m1->a8.h9<v>,0.010071,True,4000,pythia-160m


### Performance Data

In [14]:
perf_metrics = torch.load(f'results/backup/{MODEL_NAME}/nmh_backup_metrics.pt')
perf_metric_dict = {checkpoint: perf_metrics[checkpoint][PERFORMANCE_METRIC] for checkpoint in perf_metrics.keys()}

In [8]:
df_list = []
for root, dirs, files in os.walk(target_directory):
    for dir in dirs:
        folder_path = f'results/backup/{dir}/{TASK}'
        if os.path.isdir(folder_path):
            df = load_edge_scores_into_dictionary(folder_path)
            df['subfolder'] = dir  # Add the subfolder name as a new column
            df_list.append(df)
            clear_output()

combined_df = pd.concat(df_list, ignore_index=True)

NameError: name 'target_directory' is not defined

## Visualize Results

### Graph Size

In [15]:
# Group by checkpoint and subfolder and sum the number of edges
subgraph_df = all_graphs.groupby(['checkpoint', 'model']).sum().reset_index()

subgraph_df.head()


,checkpoint,model,edge,score,in_circuit
0,4000,pythia-160m,m1->m6a4.h11->a6.h6<v>a4.h8->a6.h6<v>m4->m6m1-...,0.345562,78
1,4000,pythia-160m-alldropout,m0->a7.h4<k>m0->a7.h4<v>m0->a7.h10<v>m0->m8m0-...,0.142632,47
2,4000,pythia-160m-data-seed1,m0->a8.h1<v>m0->a8.h9<v>m2->a8.h9<v>input->m0m...,0.292908,7
3,4000,pythia-160m-data-seed2,m0->a7.h10<v>m1->a8.h9<v>m0->a8.h10<v>m0->a8.h...,0.256348,22
4,4000,pythia-160m-data-seed3,m0->a8.h9<v>m0->a8.h9<k>m0->a8.h1<v>m2->a8.h9<...,0.241760,8


In [16]:
# sort this
models = subgraph_df['model'].unique().tolist()
models.sort()

In [17]:
for model in models:
    model_df = subgraph_df[subgraph_df['model'] == model].copy()
    plot_graph_metric(model_df, 'in_circuit', perf_metric_dict, f'Graph Size for {model}', y_range=1000, x_axis_col='checkpoint', log_x=True)

### Graph Transformations

In [78]:
# for model in models:
#     model_df = all_graphs[all_graphs['model'] == model].copy()
#     #weighted_ged_results = compute_weighted_ged(model_df)
#     #plot_graph_metric(weighted_ged_results, 'in_circuit', perf_metric_dict, f'Graph Size for {model}', y_range=1000, x_axis_col='checkpoint_2', log_x=True)

#     weighted_ged_results = compute_weighted_ged(model_df)
#     # Add a new column for the performance metric by mapping the checkpoint_2 values using the perf_metric_dict
#     weighted_ged_results['perf_metric'] = weighted_ged_results['checkpoint_2'].map(perf_metric_dict)

#     # Interpolate missing values
#     weighted_ged_results['perf_metric'] = weighted_ged_results['perf_metric'].interpolate(method='linear')

#     # plot weighted additions, deletions, and total weighted GED over time
#     fig = px.line(weighted_ged_results, width=1200, x='checkpoint_2', y=['weighted_additions', 'weighted_deletions', 'total_weighted_ged'], title='Weighted GED Over Time (top 100 edges)', log_x=True)

#     # Specify colors for each line
#     colors = {'weighted_additions': 'lightgreen', 'weighted_deletions': 'lightsalmon', 'total_weighted_ged': 'lightblue', 'perf_metric': 'black'}

#     # Update each trace with the specified color
#     for i, trace in enumerate(fig.data):
#         fig.data[i].update(line=dict(color=colors[trace.name]))


#     # Convert to a go.Figure to add secondary Y-axis features and add the performance metric line
#     fig.update_layout(
#         yaxis2=dict(title="Probability Difference", overlaying="y", side="right", showgrid=False)
#     )
#     fig.add_trace(
#         go.Scatter(x=weighted_ged_results['checkpoint_2'], y=weighted_ged_results['perf_metric'], name='perf_metric', mode='lines', yaxis='y2', line=dict(color=colors['perf_metric']))
#     )

#     # Optional: Update the layout if you need to adjust titles or other aesthetics
#     fig.update_layout(
#         yaxis_title="Weighted GED Metrics"
#     )

#     fig.show()

### Graph Similarity

In [18]:
for model in models:
    model_df = all_graphs[all_graphs['model'] == model].copy()
    weighted_jaccard_results = compute_weighted_jaccard_similarity(model_df)
    plot_graph_metric(weighted_jaccard_results, 'jaccard_similarity', perf_metric_dict, f'Graph Size for {model}', y_range=1, x_axis_col='checkpoint_2', log_x=True)

In [19]:
comparison_checkpoint = 5000

for model in models:
    model_df = all_graphs[all_graphs['model'] == model].copy()
    jaccard_reference_results = compute_weighted_jaccard_similarity_to_reference(model_df, comparison_checkpoint)
    plot_graph_metric(jaccard_reference_results, 'jaccard_similarity', perf_metric_dict, f'Graph Size for {model}', y_range=1, x_axis_col='checkpoint', log_x=True)

In [21]:
comparison_checkpoint = 140000

for model in models:
    model_df = all_graphs[all_graphs['model'] == model].copy()
    jaccard_reference_results = compute_weighted_jaccard_similarity_to_reference(model_df, comparison_checkpoint)
    plot_graph_metric(jaccard_reference_results, 'jaccard_similarity', perf_metric_dict, f'Graph Size for {model}', y_range=1, x_axis_col='checkpoint', log_x=True)

In [77]:
model_df['checkpoint'].unique()

array([ 4000,  5000,  6000,  7000,  8000,  9000, 10000, 11000, 12000,
       13000, 14000, 15000, 16000, 17000, 18000, 19000, 20000, 21000,
       22000, 23000, 24000, 25000, 26000, 27000, 28000, 29000, 30000,
       31000, 32000, 33000, 34000, 35000, 36000, 37000, 38000, 39000,
       40000, 41000, 42000, 43000, 44000, 45000, 46000, 47000, 48000,
       49000, 50000, 51000, 52000, 53000, 54000, 55000, 56000, 57000,
       58000, 59000, 60000, 61000, 62000, 63000, 64000, 65000, 66000,
       67000, 68000])

In [92]:
for model in models:
    model_df = all_graphs[all_graphs['model'] == model].copy()
    jaccard_reference_results = compute_ewma_weighted_jaccard_similarity(model_df, alpha=0.1)
    plot_graph_metric(jaccard_reference_results, 'ewma_change_rate', perf_metric_dict, f'EWA Graph Change Rate for {model}', y_range=1, x_axis_col='checkpoint_2', log_x=False)

In [90]:
plot_graph_metric??

Signature:
plot_graph_metric(
    df,
    metric,
    perf_metric_dict,
    title,
    y_range,
    x_axis_col='checkpoint',
    log_x=True,
)
Docstring: <no docstring>
Source:   
def plot_graph_metric(df, metric, perf_metric_dict, title, y_range, x_axis_col='checkpoint', log_x=True):
    # Add a new column for the performance metric by mapping the checkpoint_2 values using the perf_metric_dict
    df['perf_metric'] = df[x_axis_col].map(perf_metric_dict)

    # Interpolate missing values
    df['perf_metric'] = df['perf_metric'].interpolate(method='linear')

    # plot weighted additions, deletions, and total weighted GED over time
    fig = px.line(df, width=1200, x=x_axis_col, y=[metric], title=title, log_x=log_x)

    # Specify colors for each line
    colors = {metric: 'lightblue', 'perf_metric': 'black'}

    # Update each trace with the specified color
    for i, trace in enumerate(fig.data):
        fig.data[i].update(line=dict(color=colors[trace.name]))


    # Convert to a go.